In [1]:
## Libraries import

import carla, time, pygame, cv2, math, random, os, traceback, logging
# import paho.mqtt.client as mqtt
import numpy as np
import tkinter as tk
from ultralytics.models.yolo import YOLO

logging.getLogger("ultralytics").setLevel(logging.WARNING)

pygame 2.6.1 (SDL 2.28.4, Python 3.7.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [7]:
## Carla set up

try:
    client = carla.Client('localhost', 2000)
    client.set_timeout(10.0)
    world = client.get_world()
except Exception as e:
    print(f"Failed to connect to CARLA: {e}")
    exit(1)

if not all(os.path.exists(f) for f in ["yolov8n.pt"]):
    print("YOLO model files not found!")
    exit(1)

world = client.get_world()
spectator = world.get_spectator()
capture = True
stored_image = None
stored_rgb_image = None
stored_depth_image = None
backwards_only_flag = False
braking_flag = False

In [3]:
## Utliity functions

def spawn_vehicle(world, vehicle_index=0, spawn_index=0):
    blueprint_library = world.get_blueprint_library()
    vehicle_bp = blueprint_library.filter('vehicle.*')[vehicle_index]
    spawn_point = world.get_map().get_spawn_points()[spawn_index]
    vehicle = world.spawn_actor(vehicle_bp, spawn_point)
    return vehicle

def spawn_walker(world):
    blueprint_library = world.get_blueprint_library()
    
    walker_blueprints = list(blueprint_library.filter('walker.pedestrian.*'))
    
    if not walker_blueprints:
        print("No pedestrian blueprints available.")
        return None, None

    walker_bp = random.choice(walker_blueprints)
    
    if walker_bp.has_attribute("speed"):
        speed = random.uniform(0.8, 2.0)
        walker_bp.set_attribute('speed', str(speed))
    else:
        speed = 1.0
    
    spawn_points = world.get_map().get_spawn_points()
    if not spawn_points:
        print("Warning: No spawn points available for pedestrians. Cannot spawn walker.")
        return None, None
    
    spawn_point = random.choice(spawn_points)

    walker = world.try_spawn_actor(walker_bp, spawn_point)
    if walker is None:
        print("Failed to spawn walker at the chosen spawn point.")
        return None, None
    
    controller_bp = blueprint_library.find('controller.ai.walker')
    if controller_bp is None:
        print("No walker controller blueprint found.")
        return walker, None
    
    controller = world.try_spawn_actor(controller_bp, carla.Transform(), walker)
    if controller:
        controller.start()
        destination = world.get_random_location_from_navigation()
        if destination:
            controller.go_to_location(destination)
        controller.set_max_speed(speed)
    
    return walker, controller

# def callback(image):
#     image.convert(carla.ColorConverter.CityScapesPalette)
#     time.sleep(3)
#     image.save_to_disk('output/%.6d.png' % image.frame)

def render(image, display):
    if image is not None:
        array = np.frombuffer(image.raw_data, dtype=np.dtype("uint8"))
        array = np.reshape(array, (image.height, image.width, 4))
        array = array[:,:,:3]
        array = array[:,:,::-1]
        raw_image = cv2.cvtColor(array, cv2.COLOR_BGR2RGB)
        return raw_image
    
def image_processing(image, target_size):
    ih, iw = target_size
    h, w, _ = image.shape

    scale = min(iw / w, ih / h)
    nw, nh = int(scale * w), int(scale * h)

    image_resized = cv2.resize(image, (nw, nh), interpolation = cv2.INTER_LINEAR)

    image_padded = np.full((ih, iw, 3), 128.0)
    dw, dh = (iw - nw) // 2, (ih - nh) // 2
    image_padded[dh:nh+dh, dw:nw+dw, :] = image_resized

    image_padded = image_padded / 255.0
    return image_padded.astype(np.float32)

def move_spectator_to(transform, spectator, distance=5.0, x=0, y=0, z=4, yaw=0, pitch=-30, roll=0):
    back_location = transform.location - transform.get_forward_vector() * distance

    back_location.x += x
    back_location.y += y
    back_location.z += z
    transform.rotation.yaw += yaw
    transform.rotation.pitch = pitch
    transform.rotation.roll = roll

    spectator_transform = carla.Transform(back_location, transform.rotation)

    spectator.set_transform(spectator_transform)

In [4]:
## warning system (using MQTT)

def brake(vehicle):
    """Apply emergency braking to the vehicle."""
    control = carla.VehicleControl()
    control.throttle = 0.0
    control.brake = 1.0
    vehicle.apply_control(control)

# mqtt_client = mqtt.Client()
# mqtt_client.connect("localhost", 1883, 60)

def send_warning(vehicle):
    # mqtt_client.publish("warning", "Warning! Collision imminent!")
    brake(vehicle)

In [9]:
## Main utility functions

VIEW_WIDTH = tk.Tk().winfo_screenwidth()//3
VIEW_HEIGHT = tk.Tk().winfo_screenheight()//3
VIEW_FOV = 90

BB_COLOR = (248, 64, 24)

def camera_blueprint():
    """
    Returns camera blueprint.
    """
    camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')
    camera_bp.set_attribute('image_size_x', str(VIEW_WIDTH))
    camera_bp.set_attribute('image_size_y', str(VIEW_HEIGHT))
    camera_bp.set_attribute('fov', str(VIEW_FOV))

    return camera_bp

def setup_car():
    """
    Spawns actor-vehicle to be controled.
    """
    car_bp = world.get_blueprint_library().filter('vehicle.*')[0]
    location = random.choice(world.get_map().get_spawn_points())
    car = world.spawn_actor(car_bp, location)
    return car

# def callback(image):
#     """
#     Callback function to be called when new image is received.
#     """
#     image.convert(carla.ColorConverter.CityScapesPalette)
#     time.sleep(3)
#     image.save_to_disk('output/%.6d.png' % image.frame)

def set_image(img):
    global stored_image
    global capture
    if capture:
        stored_image = img
        capture = False

def set_rgb_image(image):
    """Stores the latest RGB image."""
    global stored_rgb_image
    stored_rgb_image = image

def set_depth_image(image):
    """Stores the latest Depth image."""
    global stored_depth_image
    stored_depth_image = image


def setup_camera(car):
    """
    Spawns both an RGB and a Depth camera on the vehicle.
    """
    camera_transform = carla.Transform(carla.Location(x=1.6, z=1.7), carla.Rotation(pitch=0))

    # Get camera blueprints
    blueprint_library = world.get_blueprint_library()
    
    # RGB Camera
    rgb_bp = blueprint_library.find('sensor.camera.rgb')
    rgb_bp.set_attribute('image_size_x', str(VIEW_WIDTH))
    rgb_bp.set_attribute('image_size_y', str(VIEW_HEIGHT))
    rgb_camera = world.spawn_actor(rgb_bp, camera_transform, attach_to=car)
    rgb_camera.listen(lambda image: set_rgb_image(image))

    # Depth Camera
    depth_bp = blueprint_library.find('sensor.camera.depth')
    depth_bp.set_attribute('image_size_x', str(VIEW_WIDTH))
    depth_bp.set_attribute('image_size_y', str(VIEW_HEIGHT))
    depth_camera = world.spawn_actor(depth_bp, camera_transform, attach_to=car)
    depth_camera.listen(lambda image: set_depth_image(image))

    # Camera calibration for image processing
    calibration = np.identity(3)
    calibration[0, 2] = VIEW_WIDTH / 2.0
    calibration[1, 2] = VIEW_HEIGHT / 2.0
    calibration[0, 0] = calibration[1, 1] = VIEW_WIDTH / (2.0 * np.tan(VIEW_FOV * np.pi / 360.0))
    rgb_camera.calibration = calibration
    depth_camera.calibration = calibration

    return rgb_camera, depth_camera

def get_depth_at_pixel(x, y):
    """
    Retrieves the depth value (in meters) for a given pixel (x, y).
    Assumes the stored_depth_image is in BGRA format.
    """
    if stored_depth_image is None:
        return None

    if x < 0 or x >= stored_depth_image.width or y < 0 or y >= stored_depth_image.height:
        raise ValueError("Pixel coordinates are out of bounds.")

    depth_array = np.frombuffer(stored_depth_image.raw_data, dtype=np.uint8)
    depth_array = np.reshape(depth_array, (stored_depth_image.height, stored_depth_image.width, 4))

    blue  = depth_array[y, x, 0]
    green = depth_array[y, x, 1]
    red   = depth_array[y, x, 2]

    normalized_depth = (red + green * 256 + blue * 256**2) / (256**3 - 1)

    depth_in_meters = normalized_depth * 1000.0

    return depth_in_meters

def control_car(car, backwards_only=False):
    """
    Applies control to main car based on pygame pressed keys.
    Will return True If ESCAPE is hit, otherwise False to end main loop.
    """
    control = car.get_control()

    control.brake = 0.0
    control.steer = 0.0

    move_spectator_to(car.get_transform(), spectator)
    for event in pygame.event.get():
        if event.type == pygame.KEYDOWN:
            key = event.key
            # print(key)
            if backwards_only == False:
                if key == pygame.K_ESCAPE:
                    return True # TODO:change
                if key == pygame.K_w:
                    control.reverse = False
                    control.throttle = 0.6
                if key == pygame.K_s:
                    control.reverse = True
                    control.throttle = 0.6
                if key == pygame.K_a:
                    control.throttle = 0.4
                    control.steer = -0.45
                if key == pygame.K_d:
                    control.throttle = 0.4
                    control.steer = 0.45
            elif backwards_only:
                if key == pygame.K_s:
                    control.reverse = True
                    control.throttle = 0.6
                if key == pygame.K_a:
                    control.throttle = 0.4
                    control.steer = -0.45
                if key == pygame.K_d:
                    control.throttle = 0.4
                    control.steer = 0.45

        car.apply_control(control)

def pedestrian_safety_monitoring(vehicle, results):
    """
    Monitors pedestrian safety using vehicle speed, braking distance, and depth camera data.
    
    - Stops the vehicle if a pedestrian is too close.
    - Uses depth camera to get accurate pedestrian distances.
    """
    velocity = vehicle.get_velocity()
    vehicle_speed = math.sqrt((velocity.x**2 + velocity.y**2 + velocity.z**2))

    min_safe_distance = 5 # m

    for confidence, bbox, centroid in results:
        depth = get_depth_at_pixel(int(centroid[0]), int(centroid[1]))
        if depth is None or depth > 100:
            continue

        distance = depth

        global backwards_only_flag
        global braking_flag
        if distance < min_safe_distance and vehicle_speed > 0.0 and braking_flag == False:
            control = vehicle.get_control()
            control.throttle = 0.0
            control.brake = 0.7
            vehicle.apply_control(control)
            braking_flag = True
            print("Emergency braking")
            control_car(vehicle, backwards_only=True)
            backwards_only_flag = True
        elif distance < min_safe_distance and braking_flag:
            control_car(vehicle, backwards_only=True)
            backwards_only_flag = True
        elif distance >= min_safe_distance:
            control_car(vehicle, backwards_only=False)
            braking_flag = False
            backwards_only_flag = False

def pedestrian_detection(image, model, vehicle):
    """
    Detects pedestrians in the image using YOLOv8.
    Only detections with class 'person' (class index 0) and confidence > 0.2 are considered.
    """
    results = model(image)[0]  # Run inference; get the first result object
    detections = []
    if results.boxes is not None and len(results.boxes) > 0:
        # Use recommended attributes to extract data
        boxes = results.boxes.xyxy.cpu().numpy()  # Bounding boxes: [x1, y1, x2, y2]
        confs = results.boxes.conf.cpu().numpy()    # Confidence scores
        classes = results.boxes.cls.cpu().numpy()   # Class labels
        for i in range(len(boxes)):
            if int(classes[i]) == 0 and confs[i] > 0.2:
                x1, y1, x2, y2 = boxes[i]
                bbox = (int(x1), int(y1), int(x2), int(y2))
                centroid = ((int(x1) + int(x2)) // 2, (int(y1) + int(y2)) // 2)
                detections.append((confs[i], bbox, centroid))
    pedestrian_safety_monitoring(vehicle, detections)
    return detections

In [10]:
def main():
    try:
        # Spawn vehicle
        vehicle = setup_car()
        time.sleep(2)
        move_spectator_to(vehicle.get_transform(), spectator)

        # Spawn camera
        rgb_camera, depth_camera = setup_camera(vehicle)
    
        # Spawn test pedestrians (still)
        walkers = []
        controllers = []
        for _ in range(50):
            walker, controller = spawn_walker(world)
            if walker and controller:
                walkers.append(walker)
                controllers.append(controller)

        pygame.init()
        pygame.display.set_mode((200,200))

        display = pygame.display.set_mode((VIEW_WIDTH, VIEW_HEIGHT), pygame.HWSURFACE | pygame.DOUBLEBUF)

        # input_size = 416
        
        # # labels_path = "coco.names"
        # # LABELS = open(labels_path).read().strip().split("\n")

        # weights_path = "yolov4-tiny.weights"
        # config_path = "yolov4-tiny.cfg"
        # model = cv2.dnn.readNetFromDarknet(config_path, weights_path)

        # layer_name = model.getLayerNames()
        # layer_name = [layer_name[i - 1] for i in model.getUnconnectedOutLayers()]
        # writer = None

        model = YOLO("yolov8n.pt")

        while True:
            world.tick()
            # move_spectator_to(vehicle.get_transform(), spectator)
            # Wait for both RGB and Depth images to be available
            while stored_rgb_image is None or stored_depth_image is None:
                world.tick()

            # Convert images for YOLO processing
            raw_image = render(stored_rgb_image, display)
            raw_image = cv2.cvtColor(raw_image, cv2.COLOR_BGR2RGB)
            # frame_size = raw_image.shape[:2]

            detections = pedestrian_detection(raw_image, model, vehicle)

            for conf, bbox, controid in detections:
                cv2.rectangle(raw_image, (bbox[0], bbox[1]), (bbox[2], bbox[3]), (0,255,0), 2)
            cv2.imshow("Detection", raw_image)

            pygame.display.flip()
            control_car(vehicle, backwards_only=backwards_only_flag)
    except KeyboardInterrupt:
        print('\nSimulation interrupted by user')

    finally:
        print('Cleaning up...')
        pygame.quit()
        cv2.destroyAllWindows()
        actors = world.get_actors()
        # Cleanup
        if 'rgb_camera' in locals():
            rgb_camera.destroy()
        if 'depth_camera' in locals():
            depth_camera.destroy()
        actors = world.get_actors()
        for actor in actors:
            if isinstance(actor, (carla.Vehicle, carla.Walker)):
                actor.destroy()
        print('Done.')

if __name__ == '__main__':
    try:
        main()
    except Exception as e:
        traceback.print_exc() 
        print(f'Exception occurred: {e}')

Failed to spawn walker at the chosen spawn point.
Failed to spawn walker at the chosen spawn point.
Failed to spawn walker at the chosen spawn point.
Emergency braking
Emergency braking

Simulation interrupted by user
Cleaning up...
Done.


In [ ]:
actors = world.get_actors()
for actor in actors:
    if isinstance(actor, carla.Vehicle) or isinstance(actor, carla.Walker):
        actor.destroy()